# Capybara
## Steven Ng
### Developer Skill Share
### July 7, 2016

## Integration Testing

- End-to-End Test
- Acceptance Test
- Full Stack Test
- Behavior Driven Development
- Outside-In


## References
- [Railscasts #275: How I Test](https://www.youtube.com/watch?v=AQ-Vf157Ju8)
